# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846790407483                   -0.70    4.2
  2   -7.852298977710       -2.26       -1.53    1.0
  3   -7.852646200697       -3.46       -2.52    3.2
  4   -7.852646677492       -6.32       -3.38    2.2
  5   -7.852646686128       -8.06       -4.65    1.2
  6   -7.852646686727       -9.22       -5.48    3.0
  7   -7.852646686730      -11.57       -6.23    1.2
  8   -7.852646686730      -12.86       -7.59    2.0
  9   -7.852646686730      -14.75       -7.77    3.0
 10   -7.852646686730   +    -Inf       -8.56    2.0
 11   -7.852646686730   +  -14.75       -9.22    2.0
 12   -7.852646686730   +  -14.75      -10.37    2.8
 13   -7.852646686730      -14.57      -11.35    2.8
 14   -7.852646686730   +    -Inf      -12.11    3.2


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846804771311                   -0.70           4.5
  2   -7.852526920500       -2.24       -1.63   0.80    2.2
  3   -7.852637353521       -3.96       -2.71   0.80    1.0
  4   -7.852646456413       -5.04       -3.29   0.80    2.0
  5   -7.852646674711       -6.66       -4.14   0.80    1.8
  6   -7.852646686349       -7.93       -4.81   0.80    1.5
  7   -7.852646686725       -9.43       -5.61   0.80    2.0
  8   -7.852646686730      -11.32       -6.67   0.80    1.5
  9   -7.852646686730      -12.54       -7.38   0.80    2.5
 10   -7.852646686730      -14.45       -8.18   0.80    1.5
 11   -7.852646686730   +  -14.75       -9.09   0.80    1.8
 12   -7.852646686730      -14.75       -9.57   0.80    2.5
 13   -7.852646686730   +  -14.75      -10.63   0.80    1.2
 14   -7.852646686730      -14.45      -11.82   0.80    2.2
 15   -7.852646686730   +  -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.394313e+01     3.408703e+00
 * time: 0.5118119716644287
     1     1.409176e+00     2.077444e+00
 * time: 0.7786960601806641
     2    -1.710051e+00     2.525937e+00
 * time: 0.8137130737304688
     3    -3.816168e+00     2.043623e+00
 * time: 0.8658711910247803
     4    -5.230322e+00     1.817269e+00
 * time: 0.9186639785766602
     5    -6.840058e+00     1.113156e+00
 * time: 0.9736931324005127
     6    -7.460580e+00     5.194075e-01
 * time: 1.0247859954833984
     7    -7.681192e+00     2.469104e-01
 * time: 1.061225175857544
     8    -7.761779e+00     1.649156e-01
 * time: 1.0977551937103271
     9    -7.800376e+00     1.529435e-01
 * time: 1.1328840255737305
    10    -7.827666e+00     6.345833e-02
 * time: 1.1723730564117432
    11    -7.844023e+00     4.600982e-02
 * time: 1.2093751430511475
    12    -7.850277e+00     3.097422e-02
 * time: 1.2458751201629639
    13    -7.851868e+00     2.156404e-02
 * time: 1.28137707710

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846881949815                   -0.70    5.0
  2   -7.852327624250       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686713                   -2.55
  2   -7.852646686730      -10.76       -6.01
  3   -7.852646686730   +    -Inf      -12.66


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.728053070258524e-13
|ρ_newton - ρ_scfv| = 4.953935797903657e-13
|ρ_newton - ρ_dm|   = 8.322703004693876e-10
